In [1]:
!pip install sentencepiece
!pip install transformers
!pip install accelerate -U
!pip install datasets
!pip install fairseq
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 53.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 15.2 MB/s eta 0:00:00
  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd /content/gdrive/MyDrive/Translate Project

/content/gdrive/MyDrive/Translate Project


# FineTune Step

In [1]:
import pandas as pd
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration, TrainerCallback
from datasets import Dataset
from bert_score import score as bert_score
from datasets import load_metric
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
import os

# Function to get data
def get_data(vocab_data, conversation, gramma, topik, num_x):
    """
    Get all data; if num_x is not 1, then duplicate the TOPIC data
    """

    # Read the CSV files
    vocab_data_df = pd.read_csv(vocab_data)
    conversation_df = pd.read_csv(conversation)
    topik_df = pd.read_csv(topik)
    gramma_df = pd.read_csv(gramma)

    # Combine topik_df and vocab_data_df, remove duplicates, and reset index
    data1 = pd.concat([topik_df, vocab_data_df])
    data1 = data1.drop_duplicates(subset='ko', keep='first').reset_index(drop=True)

    # Add conversation_df, remove duplicates, and reset index
    data2 = pd.concat([data1, conversation_df])
    data2 = data2.drop_duplicates(subset='ko', keep='first').reset_index(drop=True)
    
    # Add gramma_df, remove duplicates, and reset index
    data3 = pd.concat([data2,gramma_df])
    data3 = data3.drop_duplicates(subset='ko', keep='first').reset_index(drop=True)

    # If num_x is not 1, add topik_df again and reset index
    if num_x != 1:
        data3 = pd.concat([data3, topik_df]).reset_index(drop=True)

    return data3

# Preprocessing after evaluate
def preprocess_data(data):
    """
    Replace specific text in the 'vi' column of the DataFrame.
    """
    # Replace text in the 'vi' column
    data['vi'] = data['vi'].replace("Bánh gạo cay", "Bánh gạo", regex=True)
    data['vi'] = data['vi'].replace("bánh gạo cay", "bánh gạo", regex=True)

    return data

# Retrieve data and save to CSV
data = get_data(vocab_data='vocab_data.csv', conversation='conversation_data.csv', topik='topik_data.csv', gramma = "gramma_data.csv", num_x=2)
preprocessed_data = preprocess_data(data)

data.to_csv("all_data.csv", index=None)

In [3]:
import pandas as pd
df = pd.read_csv("all_data.csv")
df[df.ko.str.contains("보자기")]

,ko,vi
2646,보자기,tấm vải gói
2647,보자기는 물건을 싸는 실용적인 용도로 사용된다.,Tấm vải gói được sử dụng với mục đích thiết th...
16124,보자기,tấm vải gói
16125,보자기는 물건을 싸는 실용적인 용도로 사용된다.,Tấm vải gói được sử dụng với mục đích thiết th...


In [2]:
# Initialize tokenizer and model
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", use_fast=False, src_lang="ko", tgt_lang="vi")
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

# Function to prepare the dataset
def prep_one_dataset(filename, tokenizer):
    data = pd.read_csv(filename)
    data = data.sample(frac=1).reset_index(drop=True)
    dataset = Dataset.from_pandas(data)

    # Preprocessing function
    def preproc_func(examples):
        inputs = examples['ko']
        outputs = examples['vi']
        model_inputs = tokenizer(inputs, max_length=128, padding=True, truncation=True)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(outputs, max_length=128, padding=True, truncation=True)
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

    return dataset.map(preproc_func, batched=True, desc="Preprocessing files")

# Custom Callback for saving the model
class SaveModelCallback(TrainerCallback):
    def __init__(self, output_dir):
        self.output_dir = output_dir

    def on_epoch_end(self, args, state, control, **kwargs):
        epoch = state.epoch
        output_dir = os.path.join(self.output_dir, f'epoch_{int(epoch)}')
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

# Freeze the first 6 layers
for layer in model.model.encoder.layers[:6]:
    for param in layer.parameters():
        param.requires_grad = False


# Function to create a Trainer
def get_trainer(model, trainset, devset,outdir, batch_size=10, gradient_accumulation_steps=8, learning_rate=8e-5, weight_decay=0.02,num_epochs=4):
    training_args = Seq2SeqTrainingArguments(
        outdir,
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        gradient_accumulation_steps=gradient_accumulation_steps,
        save_total_limit=None,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        logging_dir='logs',
        # Enable mixed precision training to speed up training
        #fp16 = True
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    metric = load_metric("sacrebleu")

    # Function to compute metrics
    def compute_metrics(eval_preds):
        predictions, references = eval_preds
        if isinstance(predictions, tuple):
            predictions = predictions[0]
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_references = tokenizer.batch_decode(references, skip_special_tokens=True)
        P, R, F1 = bert_score(decoded_predictions, decoded_references, lang="vi", rescale_with_baseline=True)
        bertscore_f1 = F1.mean().item()
        bleu_score = metric.compute(predictions=decoded_predictions, references=[[r] for r in decoded_references])['score']
        return {"bleu": bleu_score, "bertscore_f1": bertscore_f1}
    
    save_callback = SaveModelCallback(output_dir=outdir)
    return Seq2SeqTrainer(
        model,
        training_args,
        train_dataset=trainset,
        eval_dataset=devset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[save_callback]
    )

# Prepare training and evaluation data
train_dataset = prep_one_dataset('all_data.csv', tokenizer)
eval_dataset = prep_one_dataset('test2.csv', tokenizer)

# Initialize and start the training process
trainer = get_trainer(model, train_dataset, eval_dataset, "finall_model2")
trainer.train()


Preprocessing files:   0%|          | 0/18195 [00:00<?, ? examples/s]

C:\Users\dobin\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Preprocessing files:   0%|          | 0/245 [00:00<?, ? examples/s]

C:\Users\dobin\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
C:\Users\dobin\AppData\Local\Temp\ipykernel_3784\1951767944.py:62: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
C:\Users\dobin\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning


Epoch,Training Loss,Validation Loss,Bleu,Bertscore F1
0,No log,0.266843,44.974302,0.920402
2,No log,0.236109,47.644898,0.928609
2,0.696100,0.227679,50.058903,0.932813
3,0.696100,0.229074,51.027336,0.934138


TrainOutput(global_step=908, training_loss=0.44454038930884543, metrics={'train_runtime': 3151.0973, 'train_samples_per_second': 23.097, 'train_steps_per_second': 0.288, 'total_flos': 1.134608354304e+16, 'train_loss': 0.44454038930884543, 'epoch': 3.99})

# Predict Step

In [51]:
model_path = "float_16model"
model = M2M100ForConditionalGeneration.from_pretrained(model_path).cuda()
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", use_fast=False, src_lang="ko", tgt_lang="vi")


def translate_korean_to_vietnamese(text):
    # Encode the Korean text
    encoded_korean = tokenizer(text, return_tensors="pt")

    # Move to GPU
    if torch.cuda.is_available():
        encoded_korean = {k: v.cuda() for k, v in encoded_korean.items()}

    # # Perform translation
    with torch.no_grad():
        translated = model.generate(**encoded_korean)

    # Decode the Vietnamese text
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

In [53]:
korean_text = '''떡볶이 맛있어요'''
# Dịch sang tiếng Việt
translated_text = translate_korean_to_vietnamese(korean_text)
translated_text

'Bánh gạo ngon.'

# Predict CSV

In [65]:
model_path = "finall_model2/epoch_3"
model = M2M100ForConditionalGeneration.from_pretrained(model_path).cuda()
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", use_fast=False, src_lang="ko", tgt_lang="vi")

# Kiểm tra xem GPU có sẵn không và đặt thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Hàm dự đoán
def predict(input_csv, output_csv):
    # Đọc dữ liệu
    data = pd.read_csv(input_csv)
    data = data.sample(20)
    # Tiền xử lý và dự đoán
    predictions = []
    for text in data['ko']:
        inputs = tokenizer(text, return_tensors="pt", max_length=128, padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        output = model.generate(**inputs)
        pred_text = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(pred_text)

    # Thêm cột dự đoán và lưu vào CSV
    data['predict'] = predictions
    data.to_csv(output_csv, index=False)

# Sử dụng hàm
predict('topik_data.csv', 'test2_predict.csv')


Using device: cuda


C:\Users\dobin\anaconda3\Lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [66]:
data = pd.read_csv("test2_predict.csv")
data

,ko,vi,predict
0,복원 기술자를 대상으로 한 3D 장치 사용 교육이 필요하다.,Đào tạo việc sử dụng thiết bị 3D cho các kỹ sư phục chế là cần thiết.,Đào tạo việc sử dụng thiết bị 3D cho các kỹ sư phục chế là cần thiết.
1,할머니가 눈 건강을 지키는 방법때문입니다.,Đó là vì phương pháp mà bà giữ gìn sức khỏe mắt.,Đó là vì phương pháp mà bà giữ gìn sức khỏe mắt.
2,특별권을 사면 놀이 시설 이용료가 할인된다.,Nếu mua vé đặc biệt thì phí sử dụng thiết bị vui chơi sẽ được giảm giá.,Nếu mua vé đặc biệt thì phí sử dụng thiết bị vui chơi sẽ được giảm giá.
3,시간이 없어서 일을 다 하지 못했습니다.,Vì không có thời gian nên tôi đã không thể làm hết công việc.,Vì không có thời gian nên tôi đã không thể làm hết công việc.
4,관중,khán giả,khán giả
5,장기적 남극 연구를 위해 예산 확보가 절실하다.,Các hoạt động nghiên cứu có hệ thống phải được thực hiện trên cơ sở hỗ trợ ngân sách lần này.,Việc đảm bảo ngân sách cho nghiên cứu nam cực lâu dài là thiếu sót.
6,저는 강아지의 주인을 만났습니다.,Tôi đã gặp chủ nhân của con chó.,Tôi đã gặp chủ nhân của con chó.
7,하지만 청소년들이 연예인을 좋아하는 것을 그렇게 부정적으로만 볼 일은 아니다.,"Tuy nhiên, không nên chỉ nhìn nhận vào việc thanh thiếu niên thích những người nổi tiếng một cách tiêu cực như vậy.","Tuy nhiên, không nên chỉ nhìn thấy việc thanh thiếu niên thích những người nổi tiếng một cách tiêu cực như vậy."
8,시민들이 역사적 가치가 높은 자산을 보존하기 시작했다.,Người dân đã bắt đầu việc bảo tồn tài sản có giá trị lịch sử cao.,Người dân đã bắt đầu bảo tồn tài sản có giá trị lịch sử cao.
9,행인,người đi đường,người đi đường


In [56]:
def translate_korean_to_vietnamese(text):
    # Encode the Korean text
    encoded_korean = tokenizer(text, return_tensors="pt")

    # Move to GPU
    if torch.cuda.is_available():
        encoded_korean = {k: v.cuda() for k, v in encoded_korean.items()}

    # # Perform translation
    with torch.no_grad():
        translated = model.generate(**encoded_korean)

    # Decode the Vietnamese text
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

In [58]:
korean_sentences = [
    "철새",
    "나는 긍정적인 마인드와 문제 해결 능력을 갖춘 리더십을 가지고 있어요.",
    "사회적 책임을 갖추어 환경 친화적이고 지속 가능한 비즈니스 모델을 촉진하고 있어요.",
    "식혜는 달고 맛있어서 사람들이 좋아해요",
    "어제 게임을 하느라고 학교 못 갔다",
    "제가 어제 본 영화는 정말 감동적이었습니다.",
    "이 책을 읽으면, 여러분도 제 생각에 동의하게 될 겁니다.",
    "그녀는 저에게 한국 문화에 대해 많이 가르쳐 줬어요.",
    "나는 내일 친구들과 함께 해변으로 갈 계획이에요.",
    "이 일을 완성하기 위해서는 더 많은 시간이 필요할 것 같습니다.",
    "만약 기회가 된다면, 저는 해외 여행을 가고 싶어요.",
    "그는 이 문제에 대해 깊이 생각하고 있는 것 같아요.",
    "저희 회사는 다음 달에 새 프로젝트를 시작할 예정입니다.",
    "그녀는 요리를 잘해서 항상 맛있는 음식을 만들어요.",
    "이 기계를 사용하는 방법을 알려주실 수 있나요?",
    "저는 어제 그녀에게 중요한 메일을 보냈습니다.",
    "만약 비가 오면, 우리는 계획을 변경해야 할 거예요.",
    "나는 이 문제에 대한 해결책을 찾는 데 도움이 필요해.",
    "그는 자신의 의견을 분명히 표현하는 데 능숙합니다.",
    "이 프로그램은 어떻게 사용하는지 설명해 주시겠어요?",
    "나는 최근에 건강에 좋은 식습관을 유지하려고 노력하고 있어요.",
    "우리 가족은 매년 여름에 바다로 여행을 가곤 해요.",
    "그녀는 매우 창의적이어서 항상 새로운 아이디어를 가지고 있어요.",
    "이 일을 어떻게 처리해야 할지 모르겠어요.",
    "저는 어릴 적부터 그림 그리는 것을 좋아했습니다."
]
for i in korean_sentences:
  print(translate_korean_to_vietnamese(i))


chim di cư
Tôi có trí óc tích cực và khả năng lãnh đạo có khả năng giải quyết vấn đề.
Chúng tôi đang thúc đẩy mô hình kinh doanh thân thiện với môi trường và có tính trách nhiệm xã hội.
Nước gạo ngọt và ngon nên mọi người thích.
Hôm qua tôi không thể đến trường vì mải chơi game.
Phim mà tôi xem hôm qua đã rất ấn tượng.
Nếu đọc cuốn sách này thì các bạn cũng sẽ đồng ý với suy nghĩ của tôi.
Cô ấy đã dạy tôi nhiều về văn hóa Hàn Quốc.
Ngày mai tôi có kế hoạch đi bờ biển cùng với bạn bè.
Có vẻ như cần nhiều thời gian hơn để hoàn thành công việc này.
Nếu có cơ hội thì tôi muốn đi du lịch nước ngoài.
Hình như anh ấy đang suy nghĩ sâu sắc về vấn đề này.
Công ty chúng tôi có kế hoạch bắt đầu dự án mới vào tháng sau.
Vì cô ấy nấu ăn giỏi nên thường xuyên làm món ăn ngon.
Anh có thể chỉ cách sử dụng máy này được không ạ?
Hôm qua tôi đã gửi email quan trọng cho cô ấy.
Nếu trời mưa thì chúng ta phải thay đổi kế hoạch.
Tôi cần giúp đỡ trong việc tìm ra giải pháp cho vấn đề này.
Anh ấy giỏi trong vi

In [50]:
korean_text = '''
어제 게임을 하느라고 오늘에 학교 못 간 뻔했다
'''
# Dịch sang tiếng Việt
translated_text = translate_korean_to_vietnamese(korean_text)
translated_text

'Hôm nay suýt nữa thì tôi không thể đi học vì mải chơi game hôm qua.'

In [3]:
import streamlit as st
from translation import translate_korean_to_vietnamese  # Import your translate function

# Title for your app
st.title("Korean to Vietnamese Translator")

# Sidebar for user input
with st.sidebar:
    st.write("Enter Korean text")
    input_text = st.text_area("Korean Text", height=200)

# Main area
if st.sidebar.button('Translate'):
    if input_text:
        # Call your translation function
        translated_text = translate_korean_to_vietnamese(input_text)

        # Display the translated text
        st.write("Translated Text (Vietnamese):")
        st.write(translated_text)
    else:
        st.write("Please enter some text to translate.")


2023-11-23 21:32:51.211 
  command:

    streamlit run C:\Users\dobin\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:
import pandas as pd
data = pd.read_csv("double_T.csv")
data[data.ko.str.contains("주시면 좋겠어요")]

,ko,vi


NameError: name 'data' is not defined

In [63]:
data[data.ko.str.contains("오늘 저녁은 집 근처 식당에서 불고기를 먹을 겁니다")]

,ko,vi


In [35]:
data[data.vi.str.contains(" cay ")]

,ko,vi
226,보통 우리가 먹는 떡볶이는 맵지만 전통 떡볶이는 맵지 않습니다.,"Thường thì bánh gạo cay chúng ta ăn rất cay, n..."
228,그래서 매운 것을 못 먹는 아이들이나 외국인도 먹을 수 있습니다.,Vì vậy trẻ em và người nước ngoài không thể ăn...
230,떡볶이는 맛이 모두 같습니다.,Tất cả bánh gạo cay đều có vị giống nhau.
231,전통 떡볶이에는 소고기를 넣습니다.,Thịt bò được thêm vào món bánh gạo cay truyền ...
232,전통 떡볶이는 맵지만 건강에 좋습니다.,Bánh gạo cay truyền thống tuy cay nhưng rất tố...
233,전통 떡볶이는 외국인이 먹기 어렵습니다.,Bánh gạo cay truyền thống rất khó ăn đối với n...
8195,저는 매운 음식을 좋아하지만 친구들은 매운 음식을 잘 못 먹어요.,Tôi thì ăn được thức ăn cay nhưng các bạn của ...
9298,의사 선생님이 음식을 싱겁게 먹으라고 했어요. 그런데 전 맵고 짠 음식이 좋아요.,Bác sĩ đã dặn tôi là phải ăn thức ăn nhạt ạ. N...
9480,혹시 메뉴 바꿀 수 있을까요? 너무 매울 것 같아서 안 매운 걸로 먹으려고요.,Liệu tôi có thể thay đổi thực đơn được không ạ...
9483,그럼 된장찌개로 바꿔 주세요. 된장찌개는 안 맵죠?,Nếu vậy hãy đổi sang món canh đậu tương giúp t...


In [4]:

# Thiết lập số hàng tối đa để hiển thị
pd.set_option('display.max_rows', None)

# Thiết lập số cột tối đa để hiển thị
pd.set_option('display.max_columns', None)

# Bạn cũng có thể thiết lập độ rộng tối đa của mỗi cột để hiển thị dữ liệu không bị cắt
pd.set_option('display.max_colwidth', None)

# Đối với hiển thị độ rộng tối đa của cửa sổ console
pd.set_option('display.width', None)